In [0]:
%run "../includes/configuration"

In [0]:
df_race_results = spark.read.parquet(f"{presentation_folder_path}/race_results")

In [0]:
from pyspark.sql import functions as F

In [0]:
df_driver_standings = df_race_results.groupBy("race_year", "driver_name", "driver_nationality").agg(F.sum("points").alias("total_points"), F.count(F.when(F.col("position") == 1, True)).alias("wins"))

In [0]:
display(df_driver_standings.filter("race_year = 2020"))

In [0]:
from pyspark.sql import Window
driver_ranks_spec = Window.partitionBy("race_year").orderBy(F.col("total_points").desc(), F.col("wins").desc())

df_driver_standings = df_driver_standings.withColumn("rank", F.rank().over(driver_ranks_spec))

In [0]:
display(df_driver_standings.filter("race_year = 2020"))

In [0]:
df_driver_standings.write.mode("overwrite").format("parquet").saveAsTable("f1_presentation.driver_standings")